In [67]:
import re
import pickle
import numpy as np
import preprocess
import random

In [2]:
def load_raw_data():
    """Return titles and lyrics."""
    
    titles_lyrics = preprocess.load_from_pickle()
    if titles_lyrics == None:
        preprocess.preprocess()
        titles_lyrics = preprocess.load_from_pickle()
    return titles_lyrics

In [7]:
lyrics = load_raw_data()

In [64]:
def lyrics_without_timing():
    """Return a list of lyrics, pure text, no timing"""
    _, lyrics = load_raw_data()
#     del_timing = lambda s: re.sub('\[.*\]', '', s).strip()
#     lyrics = [[del_timing(sentence) for sentence in lyric if del_timing(sentence) != '']\
#              for lyric in lyrics]
    
    del_not_chinese = lambda s: re.sub(u'[^\u4E00-\u9FA5 ]', '', s).strip()
    lyrics = [[del_not_chinese(sentence) for sentence in lyric\
               if del_not_chinese(sentence) != ''] for lyric in lyrics]
    
    lyrics = [[sentence for sentence in lyric
               if len(sentence) > 6
               and '词曲' not in sentence and '作曲' not in sentence and
               '编曲' not in sentence and '作词' not in sentence
              ]for lyric in lyrics]

    lyrics = [lyric[3:-1] for lyric in lyrics if len(lyric) > 15]
    return lyrics

In [81]:
def multi_q_a_lines(lyrics):
    """2 lists of sentences. Question and answer, respectively."""
    q, a = [], []
    ori_len = 0
    for lyric in lyrics:
        ori_len += len(lyric)
        q_l = []
        for i in range(len(lyric) - 1):
            qlen, alen = len(lyric[i]), len(lyric[i+1])
            q_l.append(lyric[i])
            a.append(lyric[i+1])
        for i, l in enumerate(q_l):
            line = ''
            if i - 3 >= 0:
                line += q_l[i - 3] + '，'
            if i - 2 >= 0:
                line += q_l[i - 2] + '，'
            if i - 1 >= 0:
                line += q_l[i - 1] + '，'
            line += q_l[i]
            q.append(line)
    return q, a

In [91]:
if __name__ == '__main__':
#     lyrics = lyrics_without_timing()
#     random.shuffle(lyrics)
    qlines, alines = multi_q_a_lines(lyrics)
    print(len(qlines))
    with open('train.enc', 'wb') as f:
        for line in qlines[:1800000]:
            f.write((line + '\n').encode('utf8'))
    with open('train.dec', 'wb') as f:
        for line in alines[:1800000]:
            f.write((line + '\n').encode('utf8'))
    with open('test.enc', 'wb') as f:
        for line in qlines[1800000:]:
            f.write((line + '\n').encode('utf8'))
    with open('test.dec', 'wb') as f:
        for line in alines[1800000:]:
            f.write((line + '\n').encode('utf8'))

Q & A filtered 3.9630198882069894%
1950755
